In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from ete3 import Tree

%matplotlib inline

In [2]:
newick_fp = '../data/GreatApeY.noPAR.fasta.treefile'
Tree(newick_fp)

Tree node '' (0x1555513dd12)

In [3]:
x = Tree(newick_fp, format=1)
print(x)


            /-Pan_paniscus.mPanPan1_Y
         /-|
        |   \-Pan_troglodytes.mPanTro3_Y
      /-|
     |  |   /-chm13.chm13_Y
   /-|   \-|
  |  |      \-GRCh38.GRCh38_Y
  |  |
  |   \-Gorilla_gorilla.mGorGor1_Y
--|
  |--Symphalangus_syndactylus.mSymSyn1_Y
  |
  |   /-Pongo_pygmaeus.mPonPyg2_Y
   \-|
      \-Pongo_abelii.nPonAbe1_Y


In [4]:
t2t_chrY_st5_df = pd.read_csv('../data/canonical_seq_amplicon/41586_2023_6457_MOESM4_ESM.ST5.csv', skiprows=2)
t2t_chrY_st5_df.columns

Index(['Amplicon', 'Gene Name', 'Strand', 'Gene Biotype', 'Annotation used',
       'Amplicon.1', 'Gene Name.1', 'Strand.1', 'Gene Biotype.1',
       'Annotation used.1', 'CAT/Liftoff %idy', 'RefSeq/Liftoff %idy',
       'protein coding?', 'Identical gene names?', 'Chr.', 'Start', 'End',
       'Gene Name.2', 'Strand.2', 'Gene Biotype.2', 'Notes', 'Chr..1',
       'Start.1', 'End.1', 'Gene Name.3', 'Strand.3', 'Gene Biotype.3',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37'],
      dtype='object')

In [5]:
filt_st5_df = t2t_chrY_st5_df[['Amplicon', 'Gene Name', 'Annotation used', 'Gene Biotype', 'Gene Biotype.1', 'Gene Biotype.2', 'Gene Biotype.3', 'Chr.', 'Start', 'End', 'Chr..1', 'Start.1', 'End.1']].dropna()
filt_st5_coding_df = filt_st5_df[~filt_st5_df['Gene Biotype'].str.contains('pseudogene')]
filt_st5_coding_df

,Amplicon,Gene Name,Annotation used,Gene Biotype,Gene Biotype.1,Gene Biotype.2,Gene Biotype.3,Chr.,Start,End,Chr..1,Start.1,End.1
5,TSPY,TSPY8,RefSeq/Liftoff,protein_coding,protein_coding,protein_coding,protein_coding,chrY,8943627.0,8946429.0,chrY,8943627.0,8946423.0
6,TSPY,TSPY8,RefSeq/Liftoff,protein_coding,protein_coding,protein_coding,protein_coding,chrY,8963923.0,8966719.0,chrY,8963923.0,8966719.0
7,TSPY,TSPY8,RefSeq/Liftoff,protein_coding,protein_coding,protein_coding,protein_coding,chrY,8984262.0,8987064.0,chrY,8984262.0,8987058.0
8,TSPY,TSPY2,manual curation,protein_coding,protein_coding,unprocessed_pseudogene,pseudogene,chrY,9004575.0,9007325.0,chrY,9004529.0,9007325.0
9,TSPY,TSPY4,RefSeq/Liftoff,protein_coding,protein_coding,protein_coding,protein_coding,chrY,9024834.0,9027648.0,chrY,9024834.0,9027648.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,PRY,PRY2,RefSeq/Liftoff,protein_coding,protein_coding,protein_coding,protein_coding,chrY,23326035.0,23350287.0,chrY,23326035.0,23350287.0
143,HSFY,HSFY1,RefSeq/Liftoff,protein_coding,protein_coding,protein_coding,protein_coding,chrY,19453178.0,19495418.0,chrY,19436129.0,19495418.0
144,HSFY,HSFY2,RefSeq/Liftoff,protein_coding,protein_coding,protein_coding,protein_coding,chrY,19637916.0,19680200.0,chrY,19637916.0,19697230.0
150,VCY,VCY,RefSeq/Liftoff,protein_coding,protein_coding,protein_coding,protein_coding,chrY,14892390.0,14893131.0,chrY,14892390.0,14893091.0


In [6]:
# Generate an annotation file here ... 
chroms = []
starts = []
ends = []

for x, c1, c2, s1, s2, e1, e2 in zip(filt_st5_coding_df['Annotation used'], filt_st5_coding_df['Chr.'], filt_st5_coding_df['Chr..1'], filt_st5_coding_df['Start'], filt_st5_coding_df['Start.1'], filt_st5_coding_df['End'], filt_st5_coding_df['End.1']):
    if x == 'CAT/Liftoff':
        chroms.append(c1)
        starts.append(int(s1))
        ends.append(int(e1))
    else:
        chroms.append(c1)
        starts.append(int(s1))
        ends.append(int(e1))

filt_st5_coding_df['chrom'] = chroms
filt_st5_coding_df['start'] = starts
filt_st5_coding_df['end'] = ends
filt_st5_coding_df.rename(columns = {'start': "START", "end": "END", "Amplicon": "TYPE"}, inplace=True)

filt_st5_coding_df[['START', 'END', 'TYPE']].to_csv('../data/canonical_seq_amplicon/chm13v2.chrY.protein_coding.ampliconic.tab', sep="\t", header=True, index=None)

/tmp/ipykernel_621041/2059992562.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_st5_coding_df['chrom'] = chroms
/tmp/ipykernel_621041/2059992562.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_st5_coding_df['start'] = starts
/tmp/ipykernel_621041/2059992562.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s